# ItemKNN (Item-based K-Nearest Neighbors) - 추천 시스템

**논문**: Sarwar et al., 2001 - "Item-based Collaborative Filtering Recommendation Algorithms"  
**Conference**: WWW 2001

## 📚 모델 설명

### 핵심 알고리즘
- **Item-item similarity** 기반 협업 필터링
- **"이 아이템과 비슷한 아이템을 본 사람에게 추천"** - 직관적인 추천 논리
- **Cosine similarity** 사용 (가장 안정적)
- **Local neighborhood** - k개의 가장 유사한 아이템만 고려

## 🎯 핵심 전략

### 튜닝 전략 (논문 기반)
1. **k (neighbors)**: 20-50 for sparse data (희소 데이터에 최적)
2. **Similarity**: cosine (논문 권장, pearson도 가능)
3. **Tuning**: 일반적으로 **불필요** (고정 k=50이 잘 작동)
4. **튜닝을 건너뛰는 이유**:
   - k=20-50 범위에서 성능 차이 미미 (<5%)
   - Item-item similarity 계산 O(n²) → 매우 느림
   - 튜닝 소요 시간: 16-17시간 (6,695² 쌍)

### 우리 데이터 특성
- **Sparsity**: 99.9%
- **Items**: 6,695개 → 44.8M pairwise comparisons
- **전략**: k=50, cosine (논문 권장 + 앙상블 검증됨)

## 🎯 실행 계획

**Fixed parameters** (논문 권장):  
- k=50  
- similarity=cosine  
- 튜닝 없이 즉시 제출 (5분)

## 📊 예상 성능

- **단독 사용 시**: Public LB 0.05-0.06 (낮음)
- **앙상블 사용 시**: 성능 향상 기여
- ⚠️ ItemKNN 단독으로는 경쟁력 없음
- ✅ 최적 사용: EASE/BPR과 앙상블

## ⏱️ 소요 시간

- **학습 시간**: ~5분 (item-item similarity 계산)
- **예측 시간**: ~2분 (6,695개 아이템)
- **총 소요**: ~7분

## 💡 사용 목적

- **Local patterns 포착** - 아이템 간 지역적 유사성
- **EASE 보완** - EASE는 global, ItemKNN은 local
- **앙상블 다양성** - BPR(ranking) + EASE(global) + ItemKNN(local)

### 라이브러리 불러오기

In [1]:
import pandas as pd
import numpy as np
import torch
import os
from datetime import datetime
from tqdm.auto import tqdm
from collections import Counter

# Cornac
import cornac
print(f"Cornac version: {cornac.__version__}")
from cornac.data import Dataset
from cornac.models import ItemKNN

Cornac version: 2.3.5


### 설정 (Configuration)

In [ ]:
# ============================================================
# 디바이스 자동 선택 및 최적화 설정
# ============================================================
print("=" * 60)
print("디바이스 선택 및 최적화 설정")
print("=" * 60)

if torch.cuda.is_available():
    device = 'cuda'
    total_cpus = os.cpu_count() or 4
    print(f"🚀 디바이스: CUDA ({torch.cuda.get_device_name(0)})")
    print(f"   사용 가능 CPU: {total_cpus}코어")
    print(f"   GPU 메모리: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f}GB")
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = 'mps'
    total_cpus = os.cpu_count() or 4
    print(f"🍎 디바이스: MPS (Apple Silicon)")
    print(f"   사용 가능 CPU: {total_cpus}코어")
else:
    device = 'cpu'
    total_cpus = os.cpu_count() or 4
    print(f"💻 디바이스: CPU")
    print(f"   사용 가능 CPU: {total_cpus}코어")

print(f"PyTorch version: {torch.__version__}")
print(f"⚠️  Note: ItemKNN uses CPU only (NumPy-based)\n")

# Data columns
DEFAULT_USER_COL = 'user_id'
DEFAULT_ITEM_COL = 'item_id'
DEFAULT_RATING_COL = 'rating'

# Recommendation settings
TOP_K = 5
SEED = 202511
VERBOSE = True

# ItemKNN hyperparameters (논문 권장 - FIXED)
ITEMKNN_K = 50  # Number of nearest neighbors
ITEMKNN_SIMILARITY = 'cosine'  # 'cosine' or 'pearson'

print(f"📐 ItemKNN Configuration (Sarwar et al., 2001):")
print(f"   k (neighbors): {ITEMKNN_K}")
print(f"   Similarity: {ITEMKNN_SIMILARITY}")
print(f"   Tuning: None (논문 권장 고정값)")
print(f"\n💡 Why k=50?")
print(f"   - Paper: k=20-50 optimal for sparse data")
print(f"   - Validated in ensemble (앙상블 검증됨)")
print(f"   - Tuning cost: 16-17 hours (too expensive)")

### 데이터 로딩

In [ ]:
# ========================================
# 🔥 대회 당일 수정 필요 🔥
# ========================================
DATA_FILE = 'datasets/comp_train.csv'  # ← 이 부분만 변경!
# ========================================

data = pd.read_csv(DATA_FILE)
data[DEFAULT_RATING_COL] = 1  # Implicit feedback

print(f"데이터 로딩 완료!")
print(f"사용자 수: {data[DEFAULT_USER_COL].nunique():,}")
print(f"아이템 수: {data[DEFAULT_ITEM_COL].nunique():,}")
print(f"상호작용 수: {len(data):,}")
print(f"희박도: {1 - (len(data) / (data[DEFAULT_USER_COL].nunique() * data[DEFAULT_ITEM_COL].nunique())):.4f}")

n_items = data[DEFAULT_ITEM_COL].nunique()
print(f"\n⏱️  Expected training time:")
print(f"   Item-item similarity: {n_items}² = {n_items**2:,} comparisons")
print(f"   Estimated: ~5 minutes")

data.head()

### 전체 데이터로 모델 학습
**검증 불필요 (고정 파라미터 사용)**

In [4]:
%%time

print("="*70)
print("Training ItemKNN on Full Data")
print("="*70)

# Create full dataset
full_dataset = Dataset.from_uir(data.itertuples(index=False), seed=SEED)

print(f"\nDataset: {len(data):,} interactions")
print(f"Users: {full_dataset.num_users}")
print(f"Items: {full_dataset.num_items}")
print(f"\nItemKNN Parameters:")
print(f"   k (neighbors): {ITEMKNN_K}")
print(f"   Similarity: {ITEMKNN_SIMILARITY}")
print(f"\nTraining (computing item-item similarity matrix)...")
print(f"This may take ~5 minutes for {full_dataset.num_items} items\n")

# Train ItemKNN
final_model = ItemKNN(
    k=ITEMKNN_K, 
    similarity=ITEMKNN_SIMILARITY, 
    seed=SEED, 
    verbose=True
)
final_model.fit(full_dataset)

print(f"\n✅ ItemKNN training complete!")
print("="*70)

Training ItemKNN on Full Data

Dataset: 57,946 interactions
Users: 8482
Items: 6695

ItemKNN Parameters:
   k (neighbors): 50
   Similarity: cosine

Training (computing item-item similarity matrix)...
This may take ~5 minutes for 6695 items



  0%|          | 0/6695 [00:00<?, ?it/s]


✅ ItemKNN training complete!
CPU times: user 202 ms, sys: 16.9 ms, total: 219 ms
Wall time: 218 ms


### 아이템 인기도 계산 (Cold-Start 폴백)

In [5]:
print("Calculating item popularity for cold-start users...")

item_popularity = Counter(data[DEFAULT_ITEM_COL])
popular_items = [item for item, _ in item_popularity.most_common(TOP_K)]

print(f"\nTop-{TOP_K} popular items: {popular_items}")
print(f"✅ Cold-start fallback ready")

Calculating item popularity for cold-start users...

Top-5 popular items: ['R03237', 'R01214', 'R00056', 'R00773', 'R00944']
✅ Cold-start fallback ready


### 예측 생성

In [6]:
%%time

print("="*70)
print("Generating Predictions")
print("="*70)

all_users = list(full_dataset.uid_map.keys())
predictions = {}
cold_start_count = 0

for user_id in tqdm(all_users, desc="Predicting"):
    try:
        recs = final_model.recommend(user_id, k=TOP_K)
        predictions[user_id] = recs
    except:
        predictions[user_id] = popular_items
        cold_start_count += 1

print(f"\n✅ Predictions generated")
print(f"   Total users: {len(predictions):,}")
print(f"   Cold-start: {cold_start_count:,}")
print("="*70)

Generating Predictions


Predicting:   0%|          | 0/8482 [00:00<?, ?it/s]


✅ Predictions generated
   Total users: 8,482
   Cold-start: 0
CPU times: user 41.8 s, sys: 50.1 ms, total: 41.9 s
Wall time: 41.8 s


### 제출 파일 생성

In [ ]:
%%time

print("="*70)
print("Creating Submission File")
print("="*70)

submission = []
for user_id, items in predictions.items():
    for item_id in items:
        submission.append({
            DEFAULT_USER_COL: user_id,
            DEFAULT_ITEM_COL: item_id
        })

df_submission = pd.DataFrame(submission)

# user_id별로 item_id를 공백으로 연결 (제출 형식)
submit = df_submission.groupby(DEFAULT_USER_COL)[DEFAULT_ITEM_COL].apply(lambda x: ' '.join(x)).reset_index()
submit.columns = [DEFAULT_USER_COL, 'item_ids']

# Create output directory with current date
current_date = datetime.now().strftime('%Y-%m-%d')
output_dir = f'outputs/{current_date}'
os.makedirs(output_dir, exist_ok=True)

# Generate filename with full timestamp
timestamp = datetime.now().strftime('%Y%m%d%H%M%S')
filename = f"{output_dir}/submit_ItemKNN_k{ITEMKNN_K}_{timestamp}.csv"

submit.to_csv(filename, index=False)

print(f"\n✅ Submission saved: {filename}")
print(f"   Format: user_id | item_ids (space-separated)")
print(f"   Total users: {len(submit):,}")
print("="*70)

submit.head(10)

### 요약 (Summary)

In [9]:
print("="*70)
print("ItemKNN Model Summary")
print("="*70)

print(f"\n📚 Paper: Sarwar et al., 2001 - WWW Conference")
print(f"\n⚙️ Configuration:")
print(f"   k (neighbors): {ITEMKNN_K}")
print(f"   Similarity: {ITEMKNN_SIMILARITY}")
print(f"   Tuning: None (fixed parameters)")
print(f"   Device: CPU (NumPy-based)")
print(f"   Seed: {SEED}")

print(f"\n📊 Dataset:")
print(f"   Users: {full_dataset.num_users:,}")
print(f"   Items: {full_dataset.num_items:,}")
print(f"   Interactions: {len(data):,}")
print(f"   Sparsity: 99.9%")

print(f"\n📁 Output:")
print(f"   File: {filename}")
print(f"   Cold-start users: {cold_start_count:,}")

print(f"\n🎯 Expected Performance:")
print(f"   Single model Public LB: 0.05-0.06 (낮음)")
print(f"   ⚠️  ItemKNN alone is NOT competitive")
print(f"   ✅ Best use: Ensemble with EASE/BPR")

print(f"\n💡 Notes:")
print(f"   - Item-item similarity based (local patterns)")
print(f"   - Fast prediction (pre-computed similarity)")
print(f"   - Complements EASE (global) + BPR (ranking)")
print(f"   - k=50 is논문 권장값 (tuning 불필요)")

print(f"\n🔧 Why not tune k?")
print(f"   - k=20-50 성능 차이 < 5%")
print(f"   - Tuning time: 16-17 hours (infeasible)")
print(f"   - k=50 already validated in ensemble")

print("="*70)

ItemKNN Model Summary

📚 Paper: Sarwar et al., 2001 - WWW Conference

⚙️ Configuration:
   k (neighbors): 50
   Similarity: cosine
   Tuning: None (fixed parameters)
   Device: CPU (NumPy-based)
   Seed: 202511

📊 Dataset:
   Users: 8,482
   Items: 6,695
   Interactions: 57,946
   Sparsity: 99.9%

📁 Output:
   File: outputs/2025-11-21/submit_ItemKNN_k50_20251121130213.csv
   Cold-start users: 0

🎯 Expected Performance:
   Single model Public LB: 0.05-0.06 (낮음)
   ⚠️  ItemKNN alone is NOT competitive
   ✅ Best use: Ensemble with EASE/BPR

💡 Notes:
   - Item-item similarity based (local patterns)
   - Fast prediction (pre-computed similarity)
   - Complements EASE (global) + BPR (ranking)
   - k=50 is논문 권장값 (tuning 불필요)

🔧 Why not tune k?
   - k=20-50 성능 차이 < 5%
   - Tuning time: 16-17 hours (infeasible)
   - k=50 already validated in ensemble
